In [ ]:
import sys
import logging
from urllib.parse import urlparse
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import argparse

MLFLOW_SERVER_URI = "http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/"
DATA_URL = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
EXPERIMENT_NAME = "Wine-Quality"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
mlflow.set_tracking_uri(MLFLOW_SERVER_URI)
mlflow.set_registry_uri(MLFLOW_SERVER_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print("Tracking:", mlflow.get_tracking_uri())


Tracking: http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/


In [3]:
try:
    data = pd.read_csv(DATA_URL, sep=";")
except Exception:
    logger.exception("Unable to download the data")
    raise

In [4]:
def eval_metrics(actual, pred):
    rmse = float(np.sqrt(mean_squared_error(actual, pred)))
    mae = float(mean_absolute_error(actual, pred))
    r2 = float(r2_score(actual, pred))
    return rmse, mae, r2

In [ ]:
if __name__ == "__main__":
    mlflow.set_tracking_uri(MLFLOW_SERVER_URI)
    mlflow.set_registry_uri(MLFLOW_SERVER_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)

    # Data
    try:
        data = pd.read_csv(DATA_URL, sep=";")
    except Exception:
        logger.exception("Unable to download the data")
        sys.exit(1)

    train, test = train_test_split(data, test_size=0.25, random_state=42)

    train_x = train.drop(columns=["quality"])
    test_x = test.drop(columns=["quality"])

    train_y = train["quality"]
    test_y = test["quality"]


    # Model parameters
    alpha, l1_ratio = 0.7, 0.2




In [ ]:
with mlflow.start_run(run_name=f"elasticnet_a{alpha}_l{l1_ratio}"):
    # Model
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    preds = lr.predict(test_x)
    rmse, mae, r2 = eval_metrics(test_y, preds)

    print(f"Elasticnet model (alpha={alpha:.6f}, l1_ratio={l1_ratio:.6f}):")
    print(f"  RMSE: {rmse}")
    print(f"  MAE:  {mae}")
    print(f"  R2:   {r2}")

    mlflow.log_params({"alpha": alpha, "l1_ratio": l1_ratio})
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    signature = infer_signature(train_x, lr.predict(train_x))
    input_example = train_x.iloc[:5]

    mlflow.sklearn.log_model(
        sk_model=lr,
        name="model",
        signature=signature,
        input_example=input_example,
        registered_model_name="ElasticnetWineModel",
    )

    run = mlflow.active_run()
    base = MLFLOW_SERVER_URI.rstrip("/")
    print(f"🏃 View run at: {base}/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"🧪 View experiment at: {base}/#/experiments/{run.info.experiment_id}")

Elasticnet model (alpha=0.700000, l1_ratio=0.200000):
  RMSE: 0.7274527083519514
  MAE:  0.5866230480990499
  R2:   0.14453153965976995


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
2025/10/08 15:37:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 3
Created version '3' of model 'ElasticnetWineModel'.


🏃 View run at: http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/#/experiments/1/runs/be6bf7d8c64c4d93b9de9bad1e116006
🧪 View experiment at: http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/#/experiments/1
🏃 View run elasticnet_a0.7_l0.2 at: http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/#/experiments/1/runs/be6bf7d8c64c4d93b9de9bad1e116006
🧪 View experiment at: http://ec2-3-121-219-175.eu-central-1.compute.amazonaws.com:5000/#/experiments/1
